# Script connects to metabase and exports data #

## Import required packages and pass ini file ##
metabasepy comes from https://github.com/mertsalik/metabasepy

Todo: package script with dependancies


In [ ]:
## check if packages are present, if not install them with pip

In [ ]:
import pkg_resources
import sys
req_packages = ['metabasepy','pandas','arcgis','configparser','os']

installed_packages = pkg_resources.working_set
installed_packages_list = sorted([(i.key) for i in installed_packages])
for pkg in req_packages:
    if pkg in installed_packages_list:
        print(("Found {}! Good to go!").format(pkg))
    else:
        print(("Cant find {}! Install {} before continuing!").format(pkg,pkg))



In [ ]:
## ini file must contain [metabase] header, username, password, url, databaseid. 
## see template in repo for example

In [ ]:
from metabasepy import Client,MetabaseTableParser
import configparser
import pandas as pd
import arcgis
from arcgis.features import GeoAccessor
import os

ini = r""

In [ ]:
## getpanas_sdf_exportgdb grabs all data in databaseid ini 
## currently configured to export to geodatabase
## returns up to 1,000,000 rows to gdb

In [ ]:
def getpandas_sdf_exportgdb(in_ini,out_loc,out_name,data_id):
    final_out=os.path.join(out_loc,out_name)
    config=configparser.ConfigParser()
    config.read(in_ini)
    if config.has_section('metabase')==True:
        try:
            user = config.get('metabase','username')
            passw = config.get('metabase','password')
            metaurl = config.get('metabase','url')
            
            cli = Client(user,passw,metaurl)
            cli.authenticate()
            
            json_result = cli.cards.download(data_id,format='json')
            df=pd.DataFrame(json_result)
            sdf=GeoAccessor.from_xy(df,'Longitude','Latitude', sr=4326)
            sdf.spatial.to_featureclass(final_out)
        except:
            print("Something wrong with INI login")    
    else:
        print("Metabase section not present in INI")    


In [ ]:
## getpandas_sdf returns all data found in databaseid ini as a spatially enabled pandas dataframe
## returns up to 1,000,000 rows

In [ ]:
def getpandas_sdf(in_ini,data_id):
    config=configparser.ConfigParser()
    config.read(in_ini)
    if config.has_section('metabase')==True:
        try:
            user = config.get('metabase','username')
            passw = config.get('metabase','password')
            metaurl = config.get('metabase','url')
            
            cli = Client(user,passw,metaurl)
            cli.authenticate()
            
            json_result = cli.cards.download(data_id,format='json')
            df=pd.DataFrame(json_result)
            sdf=GeoAccessor.from_xy(df,'Longitude','Latitude', sr=4326)
            return sdf
        except:
            print("Something wrong with INI login")    
    else:
        print("Metabase section not present in INI")    


In [ ]:
## get pandas with query and returns data from dataset. this id used for this is different than the getpandas_sdf function. dataset id is different than question id.
## major flaw: only returns 2000 rows.
## can be used to return counts or summary information

In [ ]:
def getpandas_with_query(in_ini,sql,databaseid):
    config=configparser.ConfigParser()
    config.read(in_ini)
    if config.has_section('metabase')==True:
        try:
            user = config.get('metabase','username')
            passw = config.get('metabase','password')
            metaurl = config.get('metabase','url')
                        
            cli = Client(user,passw,metaurl)
            cli.authenticate()
            
            result = cli.dataset.post(databaseid, query=sql)

            data_table = MetabaseTableParser.get_table(metabase_response=result)
            
            cols= []
            for name in data_table.__dict__['cols']:
                cols.append(name['display_name'])

            df = pd.DataFrame(data_table.__dict__['rows'], columns = cols )
            return df
        except:
            print("Something wrong with INI login")    
    else:
        print("Metabase section not present in INI")    
          